In [1]:
import sys
libpath = "/home/pablo/tesis/tsm-pipeline/code"
if libpath not in sys.path:
    sys.path.append(libpath)
    
%load_ext autoreload
%autoreload 2

import docker
docker_client = docker.from_env()

# Configura todo lo necesario en el ambiente
from scripts.xxx import run_tsm, ExperimentSettings, read_dbs_dataset
from scripts.evaluate import evaluate, EvaluationSettings
from os.path import join as path_join

defaults = {
    "bash_config_path": "/home/pablo/tesis/tsm-pipeline/code/scripts/config.sh",
}
defaults_evalute = {
    "cache_root":"/home/pablo/dbcache",    
}

In [2]:
tainted_path = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/tesis/tainted_path_top_100.txt")
len(tainted_path)

100

In [3]:
from sklearn.model_selection import train_test_split
from numpy.random import RandomState

# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

train, test = train_test_split(tainted_path, train_size=.5, shuffle=True, random_state=rand)

Calculo un split de 50/50 con tainted path top 100

In [4]:
results_dir = "/home/pablo/results/seldon_tainted_path_1"

In [5]:
results_dir = '/home/pablo/results/seldonstar_nosql_1'

In [5]:
# training
train_settings = ExperimentSettings(
    name="seldon_tainted_path_1",
    query_type="path",
    project_list=train,
    results_dir=results_dir,
    **defaults
)

run_tsm(docker_client, train_settings, block=True)

2023-04-30 17:27:44,277 - INFO - creating /home/pablo/results/seldon_tainted_path_1 dir. Omitting error if already existing
2023-04-30 17:27:44,278 - INFO - creating /tmp/log dir. Omitting error if already existing
2023-04-30 17:27:44,647 - INFO - running at container 63a56d178bfae86fee1dae5df06b2867f550d30b20c019137e2c5320516c5f25. Use `docker logs 63a56d178bfae86fee1dae5df06b2867f550d30b20c019137e2c5320516c5f25 --tail 10 --follow` to follow progress
2023-04-30 19:20:40,063 - INFO - running combine scores
g: /home/pablo/results/seldon_tainted_path_1/*/TaintedPathWorse-*/reprScores.txt


In [6]:
!cat /home/pablo/results/seldon_tainted_path_1/averaged-results.csv|wc -l

129


In [9]:
# evaluation
external_predicate_file = path_join(results_dir, 'averaged-results.csv')
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql",
    cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/TaintedPathWorseTSM.ql",
    external_predicate_file=external_predicate_file,
    **defaults_evalute,
)

print("Evaluating worse with %d dbs" % (len(test)))
evaluate(
    settings=worse_settings,
    output_dir=path_join(results_dir, 'worse'),
    dbs=test,
)

v0_settings = EvaluationSettings(
    search_path="/home/pablo/codeqlv0/javascript/ql/lib",
    cli_version="2.10.5",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/PathEvaluation.ql",
    **defaults_evalute,
)

print("Evaluating v0 with %d dbs" % (len(test)))
evaluate(
    settings=v0_settings,
    output_dir=path_join(results_dir, 'v0'),
    dbs=test,
)

Evaluating worse with 50 dbs
2023-05-01 11:59:31,083 - INFO - creating /home/pablo/results/seldon_tainted_path_1/worse dir. Omitting error if already existing
2023-05-01 11:59:32,567 - INFO - Starting processing with 4 processes
2023-05-01 11:59:33,091 - INFO - Evaluating GitbookIO - gitbook
2023-05-01 11:59:33,091 - INFO - Evaluating alphagov - spotlight
2023-05-01 11:59:33,091 - INFO - Evaluating alanshaw - david-www
2023-05-01 11:59:33,091 - INFO - Evaluating alumna - liven
2023-05-01 12:00:00,875 - INFO - Evaluating Lallassu - ParrotHunt
2023-05-01 12:00:01,632 - INFO - Evaluating IBM - predictive-market-using-arria
2023-05-01 12:00:01,891 - INFO - Evaluating Manouchehri - COMP2406-Assignment-2
2023-05-01 12:00:18,251 - INFO - Evaluating MrRio - jsPDF
2023-05-01 12:00:24,287 - INFO - Evaluating cgiffard - Behaviour-Assertion-Sheets
2023-05-01 12:00:24,595 - INFO - Evaluating DimiMikadze - react-native-game
2023-05-01 12:00:29,842 - INFO - Evaluating SheetJS - sheetjs
2023-05-01 12:

In [10]:
from scripts.calculate_scores import calculate_scores

calculate_scores(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
)

(0.0, 0.0, 0.0)